# Song Recommender Project

The goal of this notebook is to develop a new Song Recommender product based on Billboard - The Hot 100 list: (https://www.billboard.com/charts/hot-100/). 

When the user enters the name of a song included in the hot list, the Song Recommender will suggest another song from the hot list.

In case the song entered by the user is not included in the Billboard list anymore, the Song Recommender will conect with Spotify in order to find another song with the same features of the one entered by the user, and recommend it. 

Song Recommender will base its recommendations on the following features: 

* **Danceability**: Danceability describes how suitable for dancing a track is, based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable.

* **Acousticness**: A measure from 0.0 to 1.0 of whether the track is acoustic.

* **Energy**: Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy.

* **Instrumentalness**: Predicts whether a track contains no vocals. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content.

* **Liveness**: Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live.

* **Loudness**: The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track. Values typical range between -60 and 0 db.

* **Speechiness**: Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value.

* **Tempo**: The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration.

* **Valence**: A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry).


In [1]:
# Import all necessary libraries:

import numpy as np
import pandas as pd
import spotipy as sp
import config
from time import sleep
from random import randint
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
import random 
import pickle
from sklearn import datasets # sklearn comes with some toy datasets to practise
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from matplotlib import pyplot
from sklearn.metrics import silhouette_score
from IPython.display import Image
from IPython.display import display

In [2]:
#import files form "Spotify Clustering" Notebook (Notebook available in the repository)

def load(filename = "filename.pickle"): 
    try: 
        with open(filename, "rb") as f: 
            return pickle.load(f) 
    except FileNotFoundError: 
        print("File not found!")

In [3]:
# #import files form "Spotify Clustering" Notebook (Notebook available in the repository)
scaler2 = load("Model/scaler.pickle")
scaler2

C:\Users\merce\anaconda3\envs\Da_Env1\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator StandardScaler from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


StandardScaler()

In [4]:
#import files form "Spotify Clustering" Notebook (Notebook available in the repository)
kmeans2 = load("Model/kmeans_4.pickle")
kmeans2

C:\Users\merce\anaconda3\envs\Da_Env1\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator KMeans from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


KMeans(n_clusters=20, random_state=1234)

# Loading data

In [5]:

df=pd.read_csv("Data/hot100.csv") #import files form "Bilboard-The hot 100" Notebook (Notebook available in the repository)

clus_df=pd.read_csv("Data/clustered_df.csv")#import files form "Spotify Clustering" Notebook (Notebook available in the repository)

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=config.client_id,
                                                           client_secret=config.client_secret)) # Credentials to connect to Spotify


# Connect with Spotify

In [6]:
# Conect with Spotify in order to get the most important features of a song: 

#def spotify_search():

song = sp.search(q="Bohemian Rhapsody", limit=1) 
#pprint.pprint(song['tracks']['items'][0]['uri'])
#song["tracks"]["items"][0]["uri"]
sp.audio_features(song["tracks"]["items"][0]["uri"])[0]

{'danceability': 0.392,
 'energy': 0.402,
 'key': 0,
 'loudness': -9.961,
 'mode': 0,
 'speechiness': 0.0536,
 'acousticness': 0.288,
 'instrumentalness': 0,
 'liveness': 0.243,
 'valence': 0.228,
 'tempo': 143.883,
 'type': 'audio_features',
 'id': '7tFiyTwD0nx5a1eklYtX2J',
 'uri': 'spotify:track:7tFiyTwD0nx5a1eklYtX2J',
 'track_href': 'https://api.spotify.com/v1/tracks/7tFiyTwD0nx5a1eklYtX2J',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7tFiyTwD0nx5a1eklYtX2J',
 'duration_ms': 354320,
 'time_signature': 4}

In [7]:
#def spotify_search():
song = sp.search(q="hello", limit=1)
features=sp.audio_features(song["tracks"]["items"][0]["uri"])[0]

    

In [8]:
features

{'danceability': 0.578,
 'energy': 0.43,
 'key': 5,
 'loudness': -6.134,
 'mode': 0,
 'speechiness': 0.0305,
 'acousticness': 0.33,
 'instrumentalness': 0,
 'liveness': 0.0854,
 'valence': 0.288,
 'tempo': 78.991,
 'type': 'audio_features',
 'id': '62PaSfnXSMyLshYJrlTuL3',
 'uri': 'spotify:track:62PaSfnXSMyLshYJrlTuL3',
 'track_href': 'https://api.spotify.com/v1/tracks/62PaSfnXSMyLshYJrlTuL3',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/62PaSfnXSMyLshYJrlTuL3',
 'duration_ms': 295502,
 'time_signature': 4}

In [9]:
type(features)

dict

In [10]:
my_dict = sp.audio_features(song["tracks"]["items"][0]["uri"])[0] # you can provide a list of uri's

print(my_dict)
my_dict_new = { key: [my_dict[key]] for key in list(my_dict.keys()) }
#my_dict_new['name'] = [song["tracks"]["items"][0]['name']]
print(my_dict_new)

pd.DataFrame(my_dict_new)


{'danceability': 0.578, 'energy': 0.43, 'key': 5, 'loudness': -6.134, 'mode': 0, 'speechiness': 0.0305, 'acousticness': 0.33, 'instrumentalness': 0, 'liveness': 0.0854, 'valence': 0.288, 'tempo': 78.991, 'type': 'audio_features', 'id': '62PaSfnXSMyLshYJrlTuL3', 'uri': 'spotify:track:62PaSfnXSMyLshYJrlTuL3', 'track_href': 'https://api.spotify.com/v1/tracks/62PaSfnXSMyLshYJrlTuL3', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/62PaSfnXSMyLshYJrlTuL3', 'duration_ms': 295502, 'time_signature': 4}
{'danceability': [0.578], 'energy': [0.43], 'key': [5], 'loudness': [-6.134], 'mode': [0], 'speechiness': [0.0305], 'acousticness': [0.33], 'instrumentalness': [0], 'liveness': [0.0854], 'valence': [0.288], 'tempo': [78.991], 'type': ['audio_features'], 'id': ['62PaSfnXSMyLshYJrlTuL3'], 'uri': ['spotify:track:62PaSfnXSMyLshYJrlTuL3'], 'track_href': ['https://api.spotify.com/v1/tracks/62PaSfnXSMyLshYJrlTuL3'], 'analysis_url': ['https://api.spotify.com/v1/audio-analysis/62PaSfnXSMyLshYJ

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.578,0.43,5,-6.134,0,0.0305,0.33,0,0.0854,0.288,78.991,audio_features,62PaSfnXSMyLshYJrlTuL3,spotify:track:62PaSfnXSMyLshYJrlTuL3,https://api.spotify.com/v1/tracks/62PaSfnXSMyL...,https://api.spotify.com/v1/audio-analysis/62Pa...,295502,4


In [11]:
Z=pd.DataFrame(my_dict_new)

In [12]:
# We get numerical columns:
Z=Z._get_numeric_data()

In [13]:
scaler = StandardScaler()
scaler.fit(Z)
Z_scaled = scaler.transform(Z)
Z_scaled_df = pd.DataFrame(Z_scaled, columns = Z.columns)
display(Z.head())

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0.578,0.43,5,-6.134,0,0.0305,0.33,0,0.0854,0.288,78.991,295502,4


In [14]:
clusters = kmeans2.predict(Z_scaled_df)

In [15]:
clusters

array([3])

# Create the recommender function

In [16]:
#titles=df["titles"]

In [18]:
#titles

In [ ]:
choice=["is a great choice!"," is amazing!","is cool!"]
hot_song=""

def user_selection():
    hot_song=input("Please enter the title of a hot song: ")
    if hot_song in df.values:
        print(hot_song,random.choice(choice),"You might like this one from the hot list too:",random.choice(df["titles"]))
  
    else:
        song = sp.search(q=hot_song, limit=1)
        
        
        features=sp.audio_features(song["tracks"]["items"][0]["uri"])[0]
        
        
        my_dict_new = sp.audio_features(song["tracks"]["items"][0]["uri"])[0]
    
        Z=pd.DataFrame(my_dict_new,index=[0])
        Z=Z._get_numeric_data()
        Z_scaled=scaler2.transform(Z)
        Z_predict_cluster=kmeans2.predict(Z_scaled)[0]
        song_to_recommend=clus_df[clus_df.cluster==Z_predict_cluster].sample(1)
        song_to_recommend_uri=song_to_recommend.uri.values
        print(song_to_recommend)
        from IPython.display import IFrame
        track_id = "7kOJsVkJXvLQPQ9osGWeKd"
        IFrame(src=f"https://open.spotify.com/embed/track/{track_id}",
        width="320",
        height="80",
        frameborder="0",
        allowtransparency="true",
        allow="encrypted-media",
        )
        display(test_display())
        
      

       
        
                                                                   
      
       


In [ ]:
user_selection()

In [ ]:
from IPython.display import IFrame
track_id = "0ZfEeIu7CNHQhbCTiVv3cx"
IFrame(src=f"https://open.spotify.com/embed/track/{track_id}",
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
      )


In [ ]:
song = sp.search(q="hotel california", limit=1)
features=sp.audio_features(song["tracks"]["items"][0]["uri"])[0]

In [ ]:
features

In [ ]:
song = sp.search(q="2all", limit=1)
features=sp.audio_features(song["tracks"]["items"][0]["uri"])[0]

In [ ]:
features

In [ ]:
df["titles"]

In [ ]:
df
